## 問題生成器の準備

In [1]:
from tqdm.auto import tqdm
import pandas as pd
from pprint import pprint

INPUT_MAX_LEN = 512  # モデルに入力されるトークン列の最大長。最大長を超えたトークンは切り捨てられる。
OUTPUT_MAX_LEN = 128  # モデルから出力されるトークン列の最大長。最大長を超えないように文が生成されるはず。
MODEL_DIR = "./model"

In [2]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import T5ForConditionalGeneration, T5Tokenizer

# トークナイザー（SentencePiece）
fine_tune_tokenizer = T5Tokenizer.from_pretrained(MODEL_DIR, is_fast=True)

# 学習済みモデル
fine_tune_model = T5ForConditionalGeneration.from_pretrained(MODEL_DIR)

# GPUの利用有無
USE_GPU = torch.cuda.is_available()
if USE_GPU:
    fine_tune_model.cuda()

/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


## データセットの準備

In [5]:
test_dataset = "./data/test.csv"
question_data_list = []

df = pd.read_csv(test_dataset)

for index, row in df.iterrows():
    question_data_list.append((row["answer"], row["passage"], row["question"]))

## 問題生成

In [6]:
from pytorch_lightning.callbacks import early_stopping
generated_questions = []
csv_output = []

for answer, context, example in tqdm(question_data_list):
  # モデルに入力可能な形式に変換する。
  if answer == "" or context == "" or example == "": # 空行があれば処理を飛ばす
    continue

  input = f"answer: {answer.lower()} context: {context.lower()}"

  # 入力文をトークナイズする。
  tokenized_inputs = fine_tune_tokenizer.batch_encode_plus(
      [input], max_length=INPUT_MAX_LEN, truncation=True, 
      padding="longest", return_tensors="pt")

  input_ids = tokenized_inputs['input_ids']
  input_mask = tokenized_inputs['attention_mask']
  if torch.cuda.is_available():
    input_ids = input_ids.cuda()
    input_mask = input_mask.cuda()

  # 問題文を生成する。
  tokenized_outputs = fine_tune_model.generate(input_ids=input_ids, attention_mask=input_mask, 
    max_length=OUTPUT_MAX_LEN, return_dict_in_generate=True, decoder_start_token_id=0,
    # temperature=1.0,  # 生成にランダム性を入れる温度パラメータ
    num_beams=20,  # ビームサーチの探索幅
    # diversity_penalty=0.1,#1.0,  # 生成結果の多様性を生み出すためのペナルティパラメータ
    # num_beam_groups=4,  # ビームサーチのグループ
    num_return_sequences=20,  # 生成する文の数
    repetition_penalty=10.0, # 同じ文の繰り返し（モード崩壊）へのペナルティ
    # ,early_stopping=True
    )

  # 生成された問題文のトークン列を文字列に変換する。
  outputs = [fine_tune_tokenizer.decode(ids, skip_special_tokens=True, clean_up_tokenization_spaces=False) 
    for ids in tokenized_outputs.sequences]

  generated_questions.append(outputs)

  ## csv出力用
  for output in outputs:
      csv_output.append([answer, context, output])

import textwrap

for (answer, context, example), questions in zip(question_data_list, generated_questions):
  print(f"answer: {answer}")
  print("\n".join(textwrap.wrap(f"context:{context}")))
  print(f"example: {example}")
  for question in questions:
    print(f"  -> {question}")
  print()

## cacheクリア
df = pd.DataFrame(csv_output, columns = ["answer", "passage", "question"])
df.to_csv("./T5_20ver.csv")
torch.cuda.empty_cache()

  0%|          | 0/21 [00:00<?, ?it/s]

answer: 正しい
context:どのようなアルゴリズムでも、処理の流れは、順次、分岐、反復の3つの構造の組み合わせで構成されている。このような処理の流れを制
御構造という。順次とは1つ1つの処理を順番に行うことであり、分岐はある条件に応じて異なる処理を実行することである。また、反復はある条件が満た
されている間はその処理を繰り返し実行することである。
example: 次の文章は正しいか答えなさい。どのようなアルゴリズムでも、処理の流れは、順次、分岐、反復の3つの構造の組み合わせで構成されている。
  -> 次の文章は正しいか答えなさい。順次、分岐、反復の3つの構造の組み合わせで構成されている。
  -> 次の文章は正しいか答えなさい。処理の流れは順次、分岐、反復の3つの構造の組み合わせで構成されている。
  -> 次の文章は正しいですか?処理の流れは順次、分岐、反復の3つの構造の組み合わせで構成されている。
  -> 次の文章は正しいか答えなさい。順次、分岐、反復の3つの構造の組み合わせで構成されている複雑な処理の流れを制御構造という。
  -> 次の文章は正しいか答えなさい。反復はある条件に応じて異なる処理を実行することである。
  -> 次の文章は正しいですか?順次、分岐、反復の3つの構造の組み合わせで構成されている。
  -> 次の文章は正しいか答えなさい。反復とはある条件に応じて異なる処理を実行することである。
  -> 次の文章は正しいか答えよ。順次、分岐、反復の3つの構造の組み合わせで構成されている。
  -> 次の文章は正しいか答えなさい。順次、分岐、反復の3つの構造の組み合わせで構成されており、これらの処理の流れを制御構造という。
  -> 次の文章は正しいですか?処理の流れは順次、分岐、反復の3つの構造で構成されている。
  -> 次の文章は正しいか答えなさい。処理の流れは順次、分岐、反復の3つの構造で構成されており、これらの組み合わせを制御構造という。
  -> 次の文章は正しいか答えなさい。処理の流れは、順次、分岐、反復の3つの構造の組み合わせで構成されている。
  -> 次の文章は正しいか答えなさい。順次、分岐、反復の3つの構造を組み合わせた構成で構成されている。
  -> 次の文章は正しいですか?処理の流れは順次、分岐、反復の3つ